In [14]:
import pandas as pd
import numpy as np
import boto3
from botocore.config import Config

In [15]:
accomodation = pd.read_csv("accomodation_data.csv")
prefectures = pd.read_csv("Prefecture_Japanese_Area.tsv", sep="\t", thousands=",")

In [16]:
accomodation.head()

,時点,地域コード,地域,延べ宿泊者数（総数）【人泊】,注記
0,2007年1月,0,全国,22136270,NaN
1,2007年2月,0,全国,23346870,NaN
2,2007年3月,0,全国,26997200,NaN
3,2007年4月,0,全国,23729790,NaN
4,2007年5月,0,全国,25636730,NaN


In [17]:
accomodation.drop(["地域コード", "注記"], inplace=True, axis=1)
accomodation.rename(columns = {"時点": "time", "地域": "location", "延べ宿泊者数（総数）【人泊】": "accomodations"}, inplace=True)

In [18]:
accomodation.head()

,time,location,accomodations
0,2007年1月,全国,22136270
1,2007年2月,全国,23346870
2,2007年3月,全国,26997200
3,2007年4月,全国,23729790
4,2007年5月,全国,25636730


In [19]:
accomodation["time"] = accomodation["time"].str.replace('年','-').str.replace('月','')
accomodation = accomodation[~accomodation["time"].str.endswith("-")]
accomodation["time"] = pd.to_datetime(accomodation["time"].str.replace('年','-').str.replace('月',''), format='%Y-%m') 

In [20]:
accomodation

,time,location,accomodations
0,2007-01-01,全国,22136270
1,2007-02-01,全国,23346870
2,2007-03-01,全国,26997200
3,2007-04-01,全国,23729790
4,2007-05-01,全国,25636730
...,...,...,...
8059,2020-08-01,沖縄県,849500
8060,2020-09-01,沖縄県,884730
8061,2020-10-01,沖縄県,1285240
8062,2020-11-01,沖縄県,1415910


In [21]:
prefectures.head()

,Prefecture,Japanese,Area
0,Aichi,愛知県,5153.81
1,Akita,秋田県,11612.11
2,Aomori,青森県,9606.26
3,Chiba,千葉県,5156.15
4,Ehime,愛媛県,5676.44


In [22]:
assert(set(accomodation.location.to_list()) == set(prefectures.Japanese.to_list()))

In [23]:
accomodation = accomodation.merge(prefectures, left_on="location", right_on="Japanese")

In [24]:
accomodation.head()

,time,location,accomodations,Prefecture,Japanese,Area
0,2007-01-01,全国,22136270,Japan,全国,377915.0
1,2007-02-01,全国,23346870,Japan,全国,377915.0
2,2007-03-01,全国,26997200,Japan,全国,377915.0
3,2007-04-01,全国,23729790,Japan,全国,377915.0
4,2007-05-01,全国,25636730,Japan,全国,377915.0


In [25]:
accomodation.drop(["location", "Japanese"], axis=1, inplace=True)
accomodation.rename(columns = {"Prefecture": "location"}, inplace=True)

In [26]:
accomodation.to_csv("accomodation_processed.tsv", sep="\t", index=False)